In [92]:
mounted = False
try:
    import google.colab
    from google.colab import drive
    IN_COLAB = True
except:
    IN_COLAB = False

def mount_drive():
    global mounted
    drive.mount("/content/drive", force_remount=True)
    mounted = True

def open_dataset(filepath):
    if IN_COLAB:
        filepath = '/content/drive/My Drive/UNI/CE101/{}'.format(filepath)
        if not mounted:
            mount_drive()
    data = pd.read_csv(filepath)
    return data

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error



Initial set up code above

In [93]:
train_dataset = open_dataset("train.csv")
test_dataset = open_dataset("test.csv")

#Complete exploitation data analysis here

#Prints info about the variables contained within both datasets.
print(train_dataset.info())
print(test_dataset.info())

#Determines the missing values from the test dataset
# and shows which columns have at least one missing value.
missing_val = test_dataset.isnull().sum()
print('Data missing from test dataset:\n',missing_val[missing_val > 0])

#Determines the missing values from the train dataset
# and shows which columns have at least one missing value.
missing_val = train_dataset.isnull().sum()
print('Data missing from train dataset:\n',missing_val[missing_val > 0])
missing_val = missing_val[missing_val > 0]
print(missing_val.plot.bar())

In [94]:

#Describes all of the features of both datasets.
print(train_dataset.describe())
print(test_dataset.describe())


# Selects the variable that the model will determine. By convention,
# this is called y
y = train_dataset.SalePrice

# Creates a list of features to be selected from the dataset.
# This list is not complete, and all pre-processing of data should
# be completed before selecting features.
sales_features = ['LotArea', 'YearBuilt', 'BedroomAbvGr', 'FullBath', 'KitchenAbvGr', 'OverallQual','1stFlrSF', '2ndFlrSF']
X = train_dataset[sales_features]

train_X, val_X, train_y, val_y = train_test_split(X,y)
X_test = test_dataset[sales_features]

# This code will be used when selection only a certain amount of features.
#X_train = train_dataset[sales_features]
#X_test= test_dataset[sales_features]


#Prints the basic description of the selected features from both the test and train dataset.
print(train_X.describe(),'\n', X_test.describe())

In [95]:
train_X.head()

In [96]:
# Create the machine learning model linear regression
model = LinearRegression()


# train our model with the selected training features features.
model.fit(train_X, train_y)

#Test MAE of dataset with validation data
pred_val_y = model.predict(val_X)

mae = mean_absolute_error(pred_val_y, val_y)

print("The mean average error value is:", mae)

# Predict data based on our model
final_predict = model.predict(X_test)


pd.DataFrame({'Id': test_dataset.Id, 'SalePrice': final_predict}).to_csv("output.csv", index=False)

In [97]:
if IN_COLAB and mounted:
     drive.flush_and_unmount()